In [17]:
import numpy as np
import math
import cv2
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import tools
import pandas as pd
import sys
import os


from ipywidgets import interact, fixed

def normalize_image(img):
    img = img.astype(float)
    return ( img - np.min(img)) / ( np.max(img) - np.min(img))

def normL1(imgX, imgY):
    return np.abs(imgX) + np.abs(imgY)

def normL2(imgX, imgY):
    return np.sqrt( imgX * imgX + imgY * imgY)

In [18]:
def apply_contrast(input_img, contrast = 0):
#source : https://stackoverflow.com/questions/39308030/how-do-i-increase-the-contrast-of-an-image-in-python-opencv
    buf = input_img.copy()
    if contrast != 0:
        f = 131*(contrast + 127)/(127*(131-contrast))
        alpha_c = f
        gamma_c = 127*(1-f)
        
        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf


def local_normalize_image(img, ksize=30):
    # Compute the local background and foreground for the image
    kernel = np.ones( ( ksize, ksize), np.uint8)
    
    if ksize % 2 == 0:
        ksize -= 1

    img_erd = cv2.GaussianBlur( cv2.erode( img, kernel, iterations = 1).astype( float), (ksize, ksize), 0)
    img_dlt = cv2.GaussianBlur( cv2.dilate( img, kernel, iterations = 1).astype( float), (ksize, ksize), 0)
    
    return tools.saturate_cast_uint8( ( img - img_erd) * 255.0 / ( img_dlt - img_erd + 1))

In [19]:
train_path = '../../train/images/input/'
eval_path = '../../eval/images/input/'
test_path = '../../test/images/input/'

prepro_train_path = '../../preprocessed_train/images/input/'
prepro_eval_path = '../../preprocessed_eval/images/input/'
prepro_test_path = '../../preprocessed_test/images/input/'

input_paths = [(train_path, eval_path, test_path), (prepro_train_path, prepro_eval_path, prepro_test_path)]
i=0


median_kernel = 5
c = 15
ksize = 75

for path in input_paths[0]:
    output_path = input_paths[1][i]
    i = i+1
    
    for img_name in os.listdir(path):
        if img_name == '.DS_Store':
            continue
        image_path = path + img_name
        output_img_path = output_path + img_name
        
        ##Import image and characteristics:
        img = cv2.imread(image_path)
        
        img_blur = cv2.medianBlur(img, median_kernel)
        img_contrast = apply_contrast(img_blur, c)
        img_unif = local_normalize_image(img_contrast, ksize)
        
        cv2.imwrite(output_img_path, img_unif)
    

print("All images preprocessed and stored")  

All images preprocessed and stored
